In [1]:
from graphviz import Digraph
from collections import defaultdict
g_StartAddr = "gamerpcs.dll+0"
g_EndAddr = "gamerpcs.dll+17EACA"
g = Digraph()


读取反汇编信息

In [2]:
disData = defaultdict()
with open("DisResult.txt" ,"r") as f:
    a = ""
    
    for l in f:
        addr = l.split(" ")[0]
        dis = l[len(addr):]
        disData[addr] = dis
print(disData)

cnt = 1   time = 0\n', 'gamerpcs.dll+61A678': '     mov edi, dword ptr [rsp + 4]   cnt = 1   time = 0\n', 'gamerpcs.dll+61A67F': '     mov dword ptr [rsp], edi       cnt = 1   time = 0\n', 'gamerpcs.dll+61A682': '     pop qword ptr [rsp]            cnt = 1   time = 0\n', 'gamerpcs.dll+61A685': '     pop rdi                        cnt = 1   time = 0\n', 'gamerpcs.dll+61A686': '     lea esp, [rsp - 4]             cnt = 1   time = 0\n', 'gamerpcs.dll+61A68D': '     push rbp                       cnt = 1   time = 0\n', 'gamerpcs.dll+61A68E': '     mov ebp, dword ptr [rsp]       cnt = 1   time = 0\n', 'gamerpcs.dll+61A691': '     push rbp                       cnt = 1   time = 0\n', 'gamerpcs.dll+61A692': '     mov ebp, dword ptr [rsp]       cnt = 1   time = 0\n', 'gamerpcs.dll+61A695': '     mov dword ptr [rsp + 4], ebp   cnt = 1   time = 0\n', 'gamerpcs.dll+61A69C': '     mov ebp, dword ptr [rsp + 4]   cnt = 1   time = 0\n', 'gamerpcs.dll+61A6A3': '     pushfq                         cnt 

读取LastAddrInfo.txt

In [3]:
lastInsData = defaultdict(lambda:[])
with open("LastAddrInfo.txt" ,"r") as f:
    for l in f:
        sp = l.split(":")
        dllName = sp[0].split("+")[0]
        curIns = sp[0]
        allLast = sp[1][2:-3].split(",")
        for ins in allLast:
            if ins =="":
                continue
            composed = dllName + ins.strip()
            lastInsData[curIns].append(composed)
print(lastInsData)
            

 ['gamerpcs.dll+610797'], 'gamerpcs.dll+61079B': ['gamerpcs.dll+61079A'], 'gamerpcs.dll+61079C': ['gamerpcs.dll+61079B'], 'gamerpcs.dll+61079D': ['gamerpcs.dll+61079C'], 'gamerpcs.dll+61137B': ['gamerpcs.dll+5FA34C'], 'gamerpcs.dll+61137C': ['gamerpcs.dll+61137B'], 'gamerpcs.dll+611382': ['gamerpcs.dll+61137C'], 'gamerpcs.dll+611384': ['gamerpcs.dll+611382'], 'gamerpcs.dll+611387': ['gamerpcs.dll+611384'], 'gamerpcs.dll+611389': ['gamerpcs.dll+611387'], 'gamerpcs.dll+61138C': ['gamerpcs.dll+611389'], 'gamerpcs.dll+611392': ['gamerpcs.dll+61138C'], 'gamerpcs.dll+611398': ['gamerpcs.dll+611392'], 'gamerpcs.dll+61139A': ['gamerpcs.dll+611398'], 'gamerpcs.dll+61139D': ['gamerpcs.dll+61139A'], 'gamerpcs.dll+61139F': ['gamerpcs.dll+61139D'], 'gamerpcs.dll+6113A2': ['gamerpcs.dll+61139F'], 'gamerpcs.dll+6113A5': ['gamerpcs.dll+6113A2'], 'gamerpcs.dll+6113AB': ['gamerpcs.dll+6113A5'], 'gamerpcs.dll+6113B1': ['gamerpcs.dll+6113AB'], 'gamerpcs.dll+6113B3': ['gamerpcs.dll+6113B1'], 'gamerpcs.dll+

开始构建block

In [29]:
def isJxx(addr1, addr2):
    if disData[addr2].find("j") != -1:
        if disData[addr2].find("jmp") == -1:
            return True
    offset1 = int(addr1.split("+")[1],16)
    offset2 = int(addr2.split("+")[1],16)
    return abs(offset1 - offset2) > 16
def FindBlock(addr):
    if addr not in lastInsData:
        return [],[]
    rstBlock, rstNext = [], []
    
    while True:
        rstBlock.append(addr + disData[addr])
        nextInstLst = lastInsData[addr]
        if len(nextInstLst) == 0:
            rstNext = nextInstLst
            break
        if len(nextInstLst) > 1 or isJxx(addr, nextInstLst[0]):
            rstNext = nextInstLst
            break
        addr = nextInstLst[0]
    return rstBlock, nextInstLst

In [58]:
import queue
linkData = defaultdict(lambda:[])
blockAddr = "gamerpcs.dll+2EB0CF"
findEnd = False
procLst = queue.Queue()
item = {"addr": blockAddr, "depth":0}
procLst.put(item)
g = Digraph()
procedSet = set()
while True:
    cur = procLst.get()
    if cur == None:
        break
    curAddr = cur["addr"]
    curDepth = cur["depth"]
    #if curDepth == 20:
    #    break
    procedSet.add(curAddr)
    print("proc" + curAddr+" queue size:" + str(procLst.qsize()))
    r1, r2 = FindBlock(curAddr)
    r1.reverse()
    nodeStr = ""
    for x in r1:
        if x.find(g_StartAddr) != -1:
            findEnd = True
            break
        nodeStr += x + "\n"
    #nodeStr = r1[0] +"\n......\n" +r1[len(r1) - 1]
    g.node(curAddr, nodeStr, shape="box")
    if findEnd:
        break
    for x in r2:
        linkData[curAddr].append(x)
        if x not in procedSet:
            itemNew = {"addr":x, "depth":curDepth + 1}
            procLst.put(itemNew)
print(linkData)
for l in linkData:
    if len(linkData[l]) < 1000:
        for u in linkData[l]:
            g.edge(u, l)


0
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3AF7A0 queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3924AF queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerpcs.dll+3D05C6 queue size:10
procgamerp

KeyboardInterrupt: 

In [57]:
g.view()


FileNotFoundError: [Errno 2] No such file or directory: 'xdg-open': 'xdg-open'